In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datatable as dt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Read Data
Read Data - with DataTable libraray, Data Table reads the big data in short time. we are converting the datatable to dataframe.

In [ ]:
train_data = dt.fread('../input/jane-street-market-prediction/train.csv').to_pandas()

In [ ]:
type(train_data)

# Explore Data

In [ ]:
train_data.head()

In [ ]:
train_data.describe()

# Plot data for more analysis
Importing Ploting libraries to explore the data fruther

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(train_data.corr(), annot=True)

# Selecting top 20 percentile features
Too many features to find the coorelation and find the right features for predictions. let us use feature selection model from SKlearn to identify top 20 percentile features for prediction. 

In [ ]:
train_data.columns

In [ ]:
%%time
fit, axes = plt.subplots(nrows=45, ncols=3, figsize=(25,250))

for i in range(2,137):
    sns.distplot(train_data.iloc[:,i],ax=axes[(i-2)//3,(i-2)%3])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression, SelectPercentile, f_regression

Taking sample date for date ==1

In [ ]:
sample_data = train_data[train_data['date']==1]

In [ ]:
sample_data=sample_data[sample_data['weight']!=0]

In [ ]:
sample_data.dropna(axis=0,inplace=True)

# Sepreate independent & dependent variables
column name with "resp_" are the dependent variables and column name with "feature" are independent variables. in the below process, we are segregating the column names to split the data for X & y

In [ ]:
resp_cols = [c for c in sample_data.columns if 'resp_' in c]

In [ ]:
resp_cols

In [ ]:
features = [c for c in sample_data.columns if 'feature' in c]

In [ ]:
X= sample_data.drop(labels=resp_cols, axis=1)

In [ ]:
X.drop(labels=['date','weight','resp'], axis=1, inplace=True)

In [ ]:
y = sample_data.loc[:,resp_cols]

# Feature Selection process
lets identify the feature before test & train split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3,random_state=42)

In [ ]:
mi = SelectPercentile(mutual_info_regression, percentile=10)

In [ ]:
for i in y:
    #print(y[i])
    mi.fit(X_train,y_train[i])

In [ ]:
X_train=X_train[X_train.columns[mi.get_support()]]

In [ ]:
X_test=X_test[X_test.columns[mi.get_support()]]

In [ ]:
X_train.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
len=X_train.shape[1]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(len, activation='relu'))
model.add(Dense(16,activation ='relu'))
#model.add(Dropout(0.2))
model.add(Dense(8,activation='relu'))
model.add(Dense(16,activation ='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8,activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(4, activation='relu'))

In [ ]:
model.compile(optimizer='adam', loss='mse',metrics=['accuracy'] )

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
call_back = EarlyStopping(monitor='val_loss',patience=20)

In [ ]:
model.fit(X_train, y_train, verbose=2, callbacks=[call_back] ,validation_data=(X_test,y_test), epochs=300, batch_size=128)

Generating model report

In [ ]:
report = pd.DataFrame(model.history.history)

In [ ]:
report['accuracy'].max()

In [ ]:
report[['loss','val_loss']].plot()

In [ ]:
report[['accuracy','val_accuracy']].plot()

# conclusion:

above report shows the model is perfroming well with 51% accuracy. however, futher imporment on the feature selection whould help to predict better. 